In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Set the path to the dataset
dataset_path = r"C:\Users\riyad\OneDrive\Desktop\DATASETS\Rice_image"


C:\Users\riyad\AppData\Roaming\Python\Python310\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Step 1: Load and Prepare the Dataset

In [2]:
# Initialize empty lists for storing the images and labels
images = []
labels = []

# Loop over the subfolders in the dataset
for subfolder in os.listdir(dataset_path):
    subfolder_path = os.path.join(dataset_path, subfolder)
    if not os.path.isdir(subfolder_path):
        continue
  
    # Loop over the images in the subfolder
    for image_filename in os.listdir(subfolder_path):
        image_path = os.path.join(subfolder_path, image_filename)
        image = load_img(image_path, target_size=(224, 224))
        image = img_to_array(image)
        images.append(image)
        labels.append(subfolder)
 
# Convert to numpy arrays
images = np.array(images, dtype='float32') / 255.0  # Normalize to [0, 1]
labels = np.array(labels)

# Convert labels to one-hot encoding
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
labels = le.fit_transform(labels)
labels = to_categorical(labels, num_classes=len(le.classes_))

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)


Step 2: Traditional CNN Model

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(le.classes_), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)


Epoch 1/10
36/36 [==============================] - 65s 2s/step - loss: 0.7019 - accuracy: 0.7298 - val_loss: 0.1870 - val_accuracy: 0.9545
Epoch 2/10
36/36 [==============================] - 58s 2s/step - loss: 0.1972 - accuracy: 0.9298 - val_loss: 0.1644 - val_accuracy: 0.9336
Epoch 3/10
36/36 [==============================] - 59s 2s/step - loss: 0.1461 - accuracy: 0.9465 - val_loss: 0.1688 - val_accuracy: 0.9441
Epoch 4/10
36/36 [==============================] - 60s 2s/step - loss: 0.1344 - accuracy: 0.9526 - val_loss: 0.1621 - val_accuracy: 0.9650
Epoch 5/10
36/36 [==============================] - 61s 2s/step - loss: 0.1077 - accuracy: 0.9605 - val_loss: 0.2287 - val_accuracy: 0.9371
Epoch 6/10
36/36 [==============================] - 61s 2s/step - loss: 0.1065 - accuracy: 0.9596 - val_loss: 0.2024 - val_accuracy: 0.9441
Epoch 7/10
36/36 [==============================] - 60s 2s/step - loss: 0.0996 - accuracy: 0.9658 - val_loss: 0.2138 - val_accuracy: 0.9545
Epoch 8/10
36/36 [==

Step 3: Image Augmentation with Image Generator

In [5]:
# Define the ImageDataGenerator with augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Fit the generator to the training data
datagen.fit(X_train)

# Train the model using the augmented data
history_aug = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                        validation_data=(X_val, y_val), epochs=10)


Epoch 1/10
36/36 [==============================] - 62s 2s/step - loss: 1.6690 - accuracy: 0.3658 - val_loss: 1.2779 - val_accuracy: 0.4161
Epoch 2/10
36/36 [==============================] - 63s 2s/step - loss: 1.2820 - accuracy: 0.4035 - val_loss: 1.2382 - val_accuracy: 0.4161
Epoch 3/10
36/36 [==============================] - 64s 2s/step - loss: 1.2216 - accuracy: 0.4202 - val_loss: 1.1497 - val_accuracy: 0.4161
Epoch 4/10
36/36 [==============================] - 63s 2s/step - loss: 1.2076 - accuracy: 0.4351 - val_loss: 1.1328 - val_accuracy: 0.4441
Epoch 5/10
36/36 [==============================] - 62s 2s/step - loss: 1.1717 - accuracy: 0.4307 - val_loss: 1.0510 - val_accuracy: 0.5699
Epoch 6/10
36/36 [==============================] - 62s 2s/step - loss: 1.1186 - accuracy: 0.4518 - val_loss: 1.0982 - val_accuracy: 0.3671
Epoch 7/10
36/36 [==============================] - 67s 2s/step - loss: 1.0503 - accuracy: 0.4991 - val_loss: 1.0494 - val_accuracy: 0.4196
Epoch 8/10
36/36 [==

Step 4: Evaluate the Models

In [6]:
# Evaluate the traditional CNN model
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Traditional CNN - Accuracy: {accuracy * 100:.2f}%")

# Evaluate the augmented CNN model
loss_aug, accuracy_aug = model.evaluate(X_val, y_val)
print(f"Augmented CNN - Accuracy: {accuracy_aug * 100:.2f}%")



9/9 [==============================] - 4s 419ms/step - loss: 0.7954 - accuracy: 0.5769
Traditional CNN - Accuracy: 57.69%
9/9 [==============================] - 4s 424ms/step - loss: 0.7954 - accuracy: 0.5769
Augmented CNN - Accuracy: 57.69%
